<a href="https://www.kaggle.com/code/scottxchoo/3-baseline-model?scriptVersionId=145051322" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Baseline Model

This notebook was created for the purpose of organizing the contents of [the book "Machine Learning Deep Learning Problem Solving Strategy"](https://goldenrabbit.co.kr/product/must-have-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%C2%B7%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0-%EC%A0%84%EB%9E%B5/).

이 노트북은 [책 "머신러닝 딥러닝 문제해결 전략"](https://goldenrabbit.co.kr/product/must-have-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%C2%B7%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0-%EC%A0%84%EB%9E%B5/) 내용을 정리하는 목적으로 만들어졌습니다.

.

The base model means the most basic model of the skeleton. We will start with baseline models and model them in a way that will gradually improve performance.

베이스 모델이란 뼈대가 되는 가장 기본적인 모델을 의미합니다. 저희는 베이스라인 모델에서 출발해 성능을 점차 향상시키는 방향으로 모델링할 것입니다.

### Baseline Model Process
1. Load the data (데이터 불러오기)
2. Basic Feature Engineering (기본적인 피처 엔지니어링)
3. Create an evaluation index calculation function (평가지표 계산 함수 작성)
4. Train Model (모델 훈련)
5. Validate model performance (모델 성능 검증)
6. Submission (제출)

## 1. Load the data (데이터 불러오기)

Import contest data back into Pandas.

판다스로 경진대회 데이터를 다시 불러옵니다.

In [ ]:
import pandas as pd

data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

## 2. Basic Feature Engineering (기본적인 피처 엔지니어링)

Feature engineering is the act of transforming data. Typically, this transformation needs to be common to both training and test data, so we combine them before feature engineering and split them back up when we're done.

피처 엔지니어링은 데이터를 변환하는 작업입니다. 보통은 이 변환을 훈련 데이터와 테스트 데이터에 공통으로 반영해야 하기 때문에, 피처 엔지니어링 전에 두 데이터를 합쳤다가 다 끝나면 도로 나눠줍니다.

.

But before we combine the data, let's remove one outlier from the training data.

그런데 데이터를 합치기 전에 훈련 데이터에서 이상치 하나만 제거하고 가겠습니다.

### 2-1. Remove Outlier (이상치 제거)

As we saw in the point plot earlier, the data with a weather of 4 in the training data (rented at 6pm on a heavy rainy, snowy day) was an outlier, so we'll remove it.

앞서 포인트 플롯에서 확인한 결과 훈련 데이터에서 weather가 4인 데이터(폭우, 폭설이 내리는 날 저녁 6시에 대여)는 이상치였기에 제거하겠습니다.

In [ ]:
# Extract only data where weather is not 4 from training data (훈련 데이터에서 weather가 4가 아닌 데이터만 추출)
train = train[train['weather'] != 4]

### 2-2. Combine data (데이터 합치기)

To apply the same feature engineering to our training and test data, we'll combine them into one. Pandas' concat() function allows us to concatenate DataFrames along an axis.

훈련 데이터와 테스트 데이터에 같은 피처 엔지니어링을 적용하기 위해 두 데이터를 하나로 합치겠습니다. 판다스의 concat() 함수를 사용하면 축에 따라 DataFrame을 이어붙일 수 있습니다.

.

The training data consists of 10,886 rows and the test data consists of 6,493 rows. Since we removed the data with 4 weather (we have 1), we end up with 17,379 rows. Let's run the following code to see if it combines properly.

훈련 데이터는 10,886행, 테스트 데이터는 6,493행으로 구성되어 있습니다. 합치면 17,379행입니다. 앞서 weather가 4인 데이터를 제거했으니(1개 있음), 최종적으로 17,378행이 됩니다. 다음 코드를 실행해 제대로 합쳐지는지 보겠습니다.

In [ ]:
all_data_temp = pd.concat([train, test])
all_data_temp

There are 17,378 rows in total, but we only see indexes up to 6,492. The figure omits the middle, which is actually the result of counting from 0 to 10,885 and then counting from 0 to 6,492 (there's also an index in the middle that we 'removed earlier'). If you want to ignore the indexes in the original data and concatenate them, you can pass `ignore_index=True`.

총 17,378행인데 인덱스가 6,492까지 밖에 안 보입니다. 그림에서는 중간이 생략되어 있는데, 실제로는 0부터 10,885까지 매기고 다시 0부터 6,492까지 매긴 결과입니다(중간에 '앞서 제거한' 인덱스도 있습니다). 원래 데이터의 인덱스를 무시하고 이어붙이려면 `ignore_index=True`를 전달하면 됩니다.

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

The indexes are well represented from 0 to 17,377. You can also see that the casual, registered features and the count target value are missing in the test data, so they are marked as NaN (Not a Number).

인덱스가 0부터 17,377까지 잘 나타났습니다. 테스트 데이터에 casual, registered 피처와 count 타깃값이 없으므로 NaN(Not a Number)으로 표시된 것도 볼 수 있습니다.

### 2-3. Add derived features(variables) (파생 피처(변수) 추가)

In [ ]:
from datetime import datetime

# Create a date feature (날짜 피처 생성)
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])

# Create a year feature (연도 피처 생성)
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])

# Create a month feature (월 피처 생성)
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])

# Create a hour feature (시 피처 생성)
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# Create a weekday feature (요일 피처 생성)
all_data['weekday'] = all_data['date'].apply(lambda dateString :
datetime.strptime(dateString, "%Y-%m-%d").weekday())

The training data are records from the 1st to the 19th of each month, and the test data are records from the 20th of each month to the end of the month. Therefore, we don't need to use the day feature to predict the number of rentals. The minute and second features also have the same value in all records, so we don't need to use them for prediction. So we didn't create day, minute, and second as features.

훈련 데이터는 매달 1일부터 19일까지의 기록이고, 테스트 데이터는 매달 20일부터 월말까지의 기록입니다. 그러므로 대여 수량을 예측할 때 일(day) 피처는 사용할 필요가 없습니다. minute와 second 피처도 모든 기록에서 값이 같으므로 예측에 사용할 필요가 없습니다. 그래서 day, minute, second는 피처로 생성하지 않았습니다.

### 2-4. Remove unneeded features (필요 없는 피처 제거)

Now let's remove the features we don't need from the training and test data.

1. casual and registered features are not present in the test data, so we will remove them.
2. datetime and date features are also unnecessary because the datetime feature acts as an index, and the information in the date feature is contained in the other features (year, month, day).
3. We will also remove the month feature because the season feature is a large categorization of month.
4. We will also remove the windspeed feature because it has a weak correlation with the target value.


이제 훈련 데이터와 테스트 데이터에서 필요 없는 피처를 제거하겠습니다.

1. casual과 registered 피처는 테스트 데이터에 없는 피처이므로 제거하겠습니다.
2. datetime 피처는 인덱스 역할이고, date 피처가 갖는 정보는 다른 피처들(year, month, day)에도 담겨 있기 때문에 datetime과 date 피처도 필요 없습니다.
3. season 피처가 month의 대분류이기 때문에 month 피처도 제거하겠습니다.
4. windspeed 피처도 타깃값과 상관관계가 약해서 제거하겠습니다.

In [ ]:
drop_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month']

all_data = all_data.drop(drop_features, axis = 1)

We removed the features 'casual', 'registered', 'datetime', 'date', 'windspeed', and 'month'. By removing unnecessary features, we have curated all the features to use for modeling. We used insights from the EDA to distinguish between meaningful and unnecessary features. This process is called 'feature selection'.

'casual', 'registered', 'datetime', 'date', 'windspeed', 'month' 피처를 제거했습니다. 필요 없는 피처를 제거함으로써 모델링할 때 사용할 피처를 모두 선별했습니다. EDA에서 얻은 인사이트를 활용해 의미 있는 피처와 불필요한 피처를 구분한 것입니다. 이러한 과정을 'feature selection (피처 선택)'이라고 합니다.

.

Feature selection is the act of selecting only the key features that best represent the characteristics of the data for modeling. Feature selection has a huge impact on the performance of a machine learning model. If you have a lot of features that are irrelevant to predicting a target value, your model will perform poorly. This is not to say that more features are always better.

'feature selection (피처 선택)'은 모델링 시 데이터의 특징을 잘 나타내는 주요 피처만 선택하는 작업입니다. 피처 선택은 머신러닝 모델 성능에 큰 영향을 줍니다. 타깃값 예측과 관련 없는 피처가 많다면 오히려 예측 성능이 떨어집니다. 피처가 많다고 무조건 좋은 게 아니라는 말입니다.

### 2-5. Divide data (데이터 나누기)

Now that we've applied all the feature engineering, let's split the training and test data again.

모든 피처 엔지니어링을 적용했으므로 훈련 데이터와 테스트 데이터를 다시 나누겠습니다.

In [ ]:
# 1. Divide the training and test data (훈련 데이터와 테스트 데이터 나누기)
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 2. Remove the target value 'count' (타깃값 'count' 제거)
X_train = X_train.drop(['count'], axis = 1)
X_test = X_test.drop(['count'], axis = 1)

# 3. The target value (타깃값)
y = train['count']

1. If the target value is present, it is training data; if not, it is test data.
2. We removed X_train and X_train because they also contain the target value, count.
3. and the target value, train['count'], was assigned to the variable y separately.

Let's take a look at how the training data organization has changed after we're done with feature engineering.

1. 타깃값이 있으면 훈련 데이터이고, 없으면 테스트 데이터입니다.
2. 이렇게 나눈 X_train과 X_train에는 타깃값인 count도 포함돼 있어 제거했습니다.
3. 그리고 타깃값인 train['count']는 변수 y에 따로 할당했습니다.

피처 엔지니어링을 모두 마친 후 훈련 데이터 구성이 어떻게 바뀌었는지 살펴보겠습니다.

In [ ]:
X_train.head()

## 3. Create an evaluation index calculation function (평가지표 계산 함수 작성)

Training is the act of learning or practicing to improve a skill, so in order to make sure your training is working, you need a way to evaluate your target skill - a metric. So before we get into the nitty gritty of training, let's create a function to calculate the RMSLE, the metric for this contest.

훈련이란 어떠한 능력을 개선하기 위해 배우거나 단련하는 행위입니다. 따라서 훈련이 제대로 이루어졌는지 확인하려면 대상 능력을 평가할 수단, 즉 평가지표가 필요합니다. 그래서 본격적인 훈련에 앞서 본 경진대회 평가지표인 RMSLE를 계산하는 함수를 만들겠습니다.

In [ ]:
import numpy as np

def rmsle(y_true, y_pred, convertExp = True):
    # 1. Exponential Transformation (지수변환)
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    
    # 2. After log transformation, convert missing values to zero (로그변환 후 결측값을 0으로 변환)
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))
    
    # 3. Calculate RMSLE (RMSLE 계산)
    output = np.sqrt(np.mean((log_true - log_pred) ** 2))
    return output

The rmsle function returns the RMSLE number when passed the actual target value y_true and the predicted value y_pred as arguments. convertExp is a parameter that determines whether the input data should be exponentialized.

1. passing the default value of convertExp = True will exponentialize y_true and y_pred. We use exp(), a Numpy built-in function, to exponentialize because we want to use log(count) as the target value rather than count.

2. Logarithmize y_true and y_pred and convert missing values to 0. Note that the base of the np.log() function is e. The np.nan_to_num() function replaces all missing values with zeros. Also, np.log(y + 1) is sometimes simply expressed as np.log1p(y).

3. This is a straightforward implementation of the RMSLE formula in overflow. It calculates the final RMSLE number.

rmsle 함수는 실제 타깃값 y_true와 예측값 y_pred를 인수로 전달하면 RMSLE 수치를 반환한다. convertExp는 입력 데이터를 지수변환할지를 정하는 파라미터입니다.

1. 기본값인 convertExp = True를 전달하면 y_true와 y_pred를 지수변환합니다. 지수변환에는 넘파이 내장 함수인 exp()를 이용했습니다. 지수변환하는 이유는 타깃값으로 count가 아닌 log(count)를 사용하기 때문입니다.

2. y_true와 y_pred를 로그변환하고 결측값은 0으로 변환합니다. 참고로 np.log() 함수의 밑은 e입니다. np.nan_to_num() 함수는 결측값을 모두 0으로 바꾸는 기능을 합니다. 또한 np.log(y + 1)은 간단히 np.log1p(y)로 표현하기도 합니다.

3. RMSLE 공식을 넘파이로 그대로 구현한 코드입니다. RMSLE 수치를 최종적으로 계산해줍니다.

## 4. Train Model (모델 훈련)

Now that we have our data and evaluation function ready, let's create and train our model. First, we'll create our model by importing "LinearRegression", the simplest linear model provided by Scikit-learn.

데이터와 평가 함수가 준비되었으니 본격적으로 모델을 생성한 뒤 훈련시켜보겠습니다. 먼저 사이킷런이 제공하는 가장 간단한 선형 모델인 'LinearRegression'을 임포트하여 모델을 생성합니다.

In [ ]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

Next, train the model with the training data.

이어서 훈련 데이터로 모델을 훈련시킵니다.

In [ ]:
log_y = np.log(y) # Target value to log transformation (타깃값 로그변환)
linear_reg_model.fit(X_train, log_y) # Training Model (모델 훈련)

Training a linear regression model means finding the optimal linear regression coefficients for X_train, the independent variables (features), and log_y, the dependent variable (target value).
- Training: The process of finding the optimal weights (regression coefficients) given the features (independent variables) and the target value (dependent variable).
- Prediction: The process of estimating the target value given new independent variables (data) while the optimal weights are known (trained model).

In this context, EDA and Feature Engineering can be thought of as follows.
- EDA: The process of selecting features that will help in prediction and exploring appropriate modeling methods.
- Feature Engineering: The process of processing the selected features to make them suitable for training and to help improve performance.

선형 회귀 모델을 훈련한다는 것은 독립변수(피처)인 X_train과 종속변수(타깃값)인 log_y에 대응하는 최적의 선형 회귀 계수를 구한다는 의미입니다.
- 훈련 : 피처(독립변수)와 타깃값(종속변수)이 주어졌을 때 최적의 가중치(회귀계수)를 찾는 과정
- 예측 : 최적의 가중치를 아는 상태(훈련된 모델)에서 새로운 독립변수(데이터)가 주어졌을 때 타깃값을 추정하는 과정

이 맥락에서 EDA와 Feature Engineering은 다음처럼 풀어 생각할 수 있습니다.
- EDA : 예측에 도움이 될 피처를 추리고, 적절한 모델링 방법을 탐색하는 과정
- Feature Engineering : 추려진 피처들을 훈련에 적합하도록, 성능 향상에 도움되도록 가공하는 과정

## 5. Validate model performance (모델 성능 검증)

Now that we're done training, let's make a prediction and check the RMSLE value. Here's the code to make a prediction to validate the model performance.

훈련을 마쳤으니 예측을 해본 후 RMSLE 값까지 확인하겠습니다. 다음은 모델 성능 검증을 위해 예측을 수행하는 코드입니다.

In [ ]:
preds = linear_reg_model.predict(X_train)

When you run the code, the trained linear regression model predicts the target value based on the X_train features.

코드를 실행하면 훈련된 선형 회귀 모델이 X_train 피처를 기반으로 타깃값을 예측합니다.

.

I've used the training data for validation. This is just a test to see if I can train the model, predict the outcome, and get the RMSLE. Normally, you should use the training data for training, the validation data for validation, and the test data for testing. Rarely, if ever, will you use the data you used for training to predict.

그런데 검증 시 훈련 데이터를 사용했습니다. 모델을 훈련하고, 결과를 예측하고, 평가지표인 RMSLE까지 한번 구해보려고 시험 삼아 짠 것입니다. 원래는 훈련 시 훈련 데이터를 사용하고, 검증 시 검증 데이터를 사용하며, 테스트 시 테스트 데이터를 사용해야 합니다. 지금처럼 훈련 시 사용한 데이터를 예측할 때도 사용하는 경우는 거의 없습니다.

.

In fact, you can't get the RMSLE on the test data until you submit it, because you need the predicted and actual target values to get the RMSLE, and the test data doesn't have the actual target values. In this case, we usually split the training data into training and validation data to train the model on the training data and evaluate the performance of the trained model on the validation data.

사실 제출하기 전까지는 테스트 데이터로 RMSLE를 구할 수 없습니다. RMSLE를 구하려면 예측 타깃값과 실제 타깃값이 있어야 하는데 테스트 데이터에는 실제 타깃값이 없기 때문이죠. 이런 경우에 보통 훈련 데이터를 훈련용과 검증용으로 나눠서 훈련용 데이터로는 모델을 훈련하고, 검증용 데이터로는 훈련된 모델의 성능을 평가합니다.

.

Please note that this is just to show how training and prediction are implemented in code. Finally, let's evaluate how well we trained from the prediction results. We can do this by finding the RMSLE value between the target value log_y and the prediction result preds.

단지 훈련과 예측을 코드로 어떻게 구현하는지 간단히 보여주려는 것이니 참고만 해주세요. 마지막으로 예측 결과로부터 훈련이 얼마나 잘 되었는지를 평가해보겠습니다. 타깃값 log_y와 예측 결과 preds 사이의 RMSLE 값을 구하면 됩니다.

In [ ]:
print(f'선형 회귀의 RMSLE 값 : {rmsle(log_y, preds, True): .4f}')

As you can see in the output, the RMSLE value for the simple linear regression model is 1.0205.

출력 결과에서 볼 수 있듯이 단순 선형 회귀 모델의 RMSLE 값은 1.0205입니다.

## 6. Submission (제출)

Let's submit the results predicted by the baseline model, with two warning notes.

1. Make sure you're using predictions from test data. In the previous part of the model performance validation, we used training data to get the RMSLE value.
2. We need to exponentialize the predicted values, because the current predictions are log(count), not count.

베이스라인 모델로 예측한 결과를 제출해보겠습니다. 주의할 점은 두 가지입니다.

1. 테스트 데이터로 예측한 결과를 이용해야 합니다. 앞서 모델 성능 검증 과정에서는 RMSLE 값을 구해보고자 훈련 데이터를 이용했습니다.
2. 예측한 값에 지수변환을 해줘야 합니다. 현재 예측값이 count가 아니라 log(count)이기 때문입니다.

In [ ]:
linearreg_preds = linear_reg_model.predict(X_test) # Predict by test data (테스트 데이터로 예측)

submission['count'] = np.exp(linearreg_preds) # Exponential Transformation (지수변환)
submission.to_csv('submission.csv', index = False) # Save as File (파일로 저장)

The End.